In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from datetime import datetime
from datetime import timedelta
import warnings
import re
warnings.filterwarnings("ignore")

In [2]:
links = open("links1.txt", "r")
file_content = links.read()
link_list = file_content.split("\n")
links.close()


In [3]:
link_list.pop()

''

In [4]:
def trova_titoli(soup):
    titolo=soup.find(itemprop="name")
    if titolo != None:
        return(titolo['content'])
    else:
        return('NA')

In [5]:
def trova_durata(soup):
    durata=soup.find(itemprop="duration")
    if durata != None:
        raw_time=durata['content'][2:]
        raw_time=raw_time.replace('M','-')
        raw_time=raw_time.replace('S','')
        m,s=raw_time.split('-')
        duration=int(m)*60+int(s)
        return(duration)
    else:
        return('NA')

In [6]:
def trova_genere(soup):
    genere=soup.find(itemprop="genre")
    if genere != None:
        return(genere['content'])
    else:
        return('NA')

In [7]:
def trova_views(soup):
    views=soup.find(itemprop="interactionCount")
    if views != None:
        return(views['content'])
    else:
        return('NA')

In [8]:
def trova_FF(soup):
    FF=soup.find(itemprop="isFamilyFriendly")
    if FF != None:
        return(FF['content'])
    else:
        return('NA')

In [9]:
def trova_Paid(soup):
    paid=soup.find(itemprop="paid")
    if paid != None:
        return(paid['content'])
    else:
        return('NA')

In [10]:
def trova_DataPubblicazione(soup):
    publ=soup.find(itemprop="datePublished")
    if publ != None:
        return(publ['content'])
    else:
        return('NA')

In [11]:
def trova_commenti(soup):
  web_page = str(soup.body)
  search_result = re.findall(re.compile('"commentCount":{"simpleText":"[\d.]+'), web_page)
  if len(search_result) == 0:
    n_comments = 0
  else:
    n_comments = int(search_result[0][30:].replace('.', ''))
  return n_comments

In [12]:
# per ora l'output è una stringa, ad esempio: 3,5 milioni
# dopo quando abbiamo raccolto i dati vedremo come convertire le stringhe a seconda dei casi che ci saranno
def trova_iscritti(soup):
  web_page = str(soup.body)
  search_result = re.findall(re.compile('"subscriberCountText":{"accessibility":{"accessibilityData":{"label":"[\d.]+.+ iscritti"}}'), 
                             web_page)
  if len(search_result) == 0:
    subs = 0
    return subs
  else:
    subs = search_result[0][70:-3].removesuffix(' iscritti').replace('.', '')
    if subs.isnumeric():
      return int(subs)
    else:
      subs = int(subs.removesuffix(' milioni di').replace(',', ''))
      subs = int(subs * 10e6)
    
    return subs

In [13]:
def trova_like(soup):
  web_page = str(soup.body)
  search_result = re.findall(re.compile('Mi piace"}},"simpleText":"[\d.]+'), web_page)
  if len(search_result) == 0:
    likes = 0
  else:
    likes = int(search_result[0][26:].replace('.', ''))
  return likes

In [14]:
#write to file
file=open('dataset.csv','w')
file.write('titolo, genere, durata, views, n_like, n_comments, FF, paid, publ\n')

for link in tqdm(link_list[:50]): #limite sui primi due per fare una prova
    headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
    connection=requests.get(link, verify=False, stream=True, headers=headers)
    if connection.status_code==200:
        soup = BeautifulSoup(connection.text,'html.parser')
        
        titolo     = trova_titoli(soup)
        durata     = trova_durata(soup)
        genere     = trova_genere(soup)
        views      = trova_views(soup)
        n_comments = trova_commenti(soup)
        iscritti   = trova_iscritti(soup)
        n_like     = trova_like(soup)
        FF         = trova_FF(soup)
        paid       = trova_Paid(soup)
        publ       = trova_DataPubblicazione(soup)
        
        file.write(('{}, {}, {}, {}, {}, {}, {}, {}, {}\n').format(titolo, genere, durata, views, n_like, n_comments, FF, paid, publ))
    else:
        print("couldn't establish connection")
file.close()

100%|███████████████████████████████████████████| 50/50 [22:17<00:00, 26.75s/it]
